# pyAnVIL FHIR extract and QA

## overview
![image](https://user-images.githubusercontent.com/47808/102566809-16b1fc00-4095-11eb-8cf8-f78952ba0464.png)


## dependencies

> Ensure latest version of pyAnVIL installed

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip uninstall  -y pyanvil  
!{sys.executable} -m pip install   pyAnVIL==0.0.5rc5 --upgrade
!{sys.executable} -m pip show pyanvil


> Ensure PFB extract available
![image](https://user-images.githubusercontent.com/47808/99719432-21ab4980-2a61-11eb-8377-6cbd6ab156ed.png)


In [1]:
import os

AVRO_PATH = "/tmp/export_2020-11-05T23_26_49.avro"

!gsutil cp $WORKSPACE_BUCKET/export_2020-11-05T23_26_49.avro AVRO_PATH

assert os.path.isfile(AVRO_PATH), f"{AVRO_PATH} should exist. Please export PFB from https://gen3.theanvil.io/" 

Copying gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/export_2020-11-05T23_26_49.avro...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

- [1 files][226.4 MiB/226.4 MiB]                                                
Operation completed over 1 objects/226.4 MiB.                                    


# extract

> Extract all meta data, write terra, gen3 sqlite databases and dashboard summary

In [8]:
"""Extract all workspaces."""
import os
import logging
import json

from anvil.terra.reconciler import Reconciler, Entities
from anvil.util.reconciler import aggregate, DEFAULT_NAMESPACE
from anvil.transformers.fhir.transformer import FhirTransformer
from anvil.dbgap.api import get_accession, get_study


logging.basicConfig(level=logging.ERROR, format='%(asctime)s %(levelname)-8s %(message)s')
DASHBOARD_OUTPUT_PATH = "/tmp"
TERRA_SUMMARY = f"{DASHBOARD_OUTPUT_PATH}/terra_summary.json"
DASHBOARD_OUTPUT_FILE = f"{DASHBOARD_OUTPUT_PATH}/data_dashboard.json"


def harvest_workspaces(consortiums):
    """Harvest all workspaces, return list of workspace_name. Create detailed sqlite graph and summary dashboard."""
    logging.info("Starting aggregation for all AnVIL workspaces, this will take several minutes.")
    
    with open(DASHBOARD_OUTPUT_FILE, 'w') as outs:
        json.dump({'projects': [v for v in aggregate(namespace=DEFAULT_NAMESPACE,
                                                     user_project=os.environ['GOOGLE_PROJECT'],
                                                     consortium=consortiums,
                                                     avro_path=AVRO_PATH
                                                    )]}, outs)
        
    assert os.path.isfile(DASHBOARD_OUTPUT_FILE), f"{DASHBOARD_OUTPUT_FILE} should exist."
    assert os.path.isfile('/tmp/terra.sqlite'), f"'/tmp/terra.sqlite' should exist."
    
    entities = Entities(path='/tmp/terra.sqlite')
    return [workspace.name for workspace in entities.get_by_name('workspace')]    
    

def write_fhir(workspace_names):
    """Write all fhir objects."""
    entities = Entities(path='/tmp/terra.sqlite')

    for name in workspace_names:
        emitters = {}
        entity = entities.get(name)
        workspace = entity['vertex']
        logging.info(f"Transforming {name}")
        if 'subject' not in entity['edges']:
            logging.error(f"{name} missing subject edges")
            continue
        workspace._subjects = entity['edges']['subject']
        warned_missing_samples = False
        for subject in workspace.subjects:
            entity = entities.get(subject.id)
            if 'sample' not in entity['edges']:
                if not warned_missing_samples:
                    logging.warning(f"{subject.id} missing sample edges")
                warned_missing_samples = True
                continue
            subject.samples = entity['edges']['sample']
            for sample in subject.samples:
                entity = entities.get(sample.id)
                _blobs = entity['edges'].get('blob', None)
                if _blobs:
                    sample.blobs = {b['property_name']: b for b in _blobs}
        transformer = FhirTransformer(workspace=workspace)
        # namespace = workspace.attributes.workspace.namespace
        reconciler_name = workspace.attributes.reconciler_name
        for item in transformer.transform():
            for entity in item.entity():
                resourceType = entity['resourceType']
                dir_path = f"{DASHBOARD_OUTPUT_PATH}/{reconciler_name}/{name}"
                file_path = f"{dir_path}/{resourceType}.json"
                emitter = emitters.get(file_path, None)
                if emitter is None:
                    os.makedirs(dir_path, exist_ok=True)
                    emitter = open(file_path, "w")
                    logging.info(f"Writing {file_path}")
                    emitters[file_path] = emitter
                json.dump(entity, emitter, separators=(',', ':'))
                emitter.write('\n')
        for stream in emitters.values():
            stream.close()


## extract & validate

In [9]:
try:
    os.unlink('/tmp/terra.sqlite')
except Exception as e:
    print(e)
    
consortiums = (
    ('CMG', 'AnVIL_CMG_.*'),
    ('CCDG', 'AnVIL_CCDG_.*'),
    ('GTEx', '^AnVIL_GTEx_V8_hg38$'),
    ('ThousandGenomes', '^1000G-high-coverage-2019$'),
)    
workspace_names = [n for n in harvest_workspaces(consortiums)]
print(workspace_names)

['AnVIL_CMG_Broad_Muscle_KNC_WGS', 'ANVIL_CMG_Broad_Muscle_Laing_WES', 'AnVIL_CMG_Broad_Orphan_VCGS-White_WES', 'AnVIL_CMG_Broad_Muscle_Myoseq_WES', 'AnVIL_CMG_Broad_Heart_Ware_WES', 'AnVIL_CMG_Broad_Muscle_Beggs_WES', 'AnVIL_CMG_Broad_Blood_Gazda_WGS', 'AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WES', 'AnVIL_CMG_Broad_Eye_Pierce_WES', 'AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WGS', 'AnVIL_CMG_Broad_Muscle_Topf_WES', 'AnVIL_CMG_Broad_Blood_Sankaran_WES', 'AnVIL_CMG_Broad_Brain_Walsh_WES', 'AnVIL_CMG_Broad_Muscle_Kang_WES', 'AnVIL_CMG_Broad_Kidney_Hildebrandt_WES', 'AnVIL_CMG_Broad_Muscle_OGrady_WES', 'AnVIL_CMG_Broad_Brain_Gleeson_WES', 'AnVIL_CMG_UWash_GRU', 'AnVIL_CMG_Broad_Brain_Gleeson_WGS', 'AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS', 'AnVIL_CMG_Broad_Blood_Gazda_WES', 'AnVIL_CMG_Broad_Orphan_Manton_WES', 'AnVIL_CMG_Broad_Brain_Engle_WGS', 'AnVIL_CMG_Broad_Heart_PCGC-Tristani_WGS', 'AnVIL_CMG_Broad_Kidney_Pollak_WES', 'AnVIL_CMG_Broad_Orphan_Scott_WES', 'AnVIL_CMG_Broad_Muscle_KNC_WES', 'AnVIL

# QA Report 

> Show reconciliation with terra, gen3

##  Issues/Questions arising from Terra

In [13]:
# python json serializer setup

import datetime
import json
import os
from anvil.util.reconciler import flatten
import pandas as pd

def json_serial(obj):
    """JSON serializer for objects not serializable by default json code."""
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError("Type %s not serializable" % type(obj))


# validate output summary and 
assert os.path.isfile(DASHBOARD_OUTPUT_FILE), "dashboard should exist"
with open(DASHBOARD_OUTPUT_FILE, 'r') as inputs:
    dashboard_data = json.load(inputs)
    
# Flatten dashboard into tsv

(flattened, column_names) = flatten(dashboard_data['projects'])
df = pd.DataFrame(flattened)  
df.columns = column_names
# Print the data  (all rows, all columns)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# export create a tsv from dataframe
df.to_csv("/tmp/data_dashboard.tsv", sep="\t")
df

,source,workspace,accession,Bai,Bam,Crai,Cram,Md5,Tbi,Vcf,size,Project,Samples,Subject,dbgap_sample_count_mismatch,inconsistent_entityName,inconsistent_subject,missing_accession,missing_blobs,missing_samples,missing_schema,missing_sequence,missing_subjects
0,CMG,AnVIL_CMG_Broad_Muscle_KNC_WGS,phs001272.v1.p1,,,,,,,,0,1,14,14,,,,,,,,True,
1,CMG,ANVIL_CMG_Broad_Muscle_Laing_WES,phs001272.v1.p1,,,,,,,,0,1,31,31,,,,,,,,True,
2,CMG,AnVIL_CMG_Broad_Orphan_VCGS-White_WES,phs001272.v1.p1,,,,,,,,0,1,447,447,,,,,,,,True,
3,CMG,AnVIL_CMG_Broad_Muscle_Myoseq_WES,phs001272.v1.p1,,,,,,,,0,1,1280,1280,,,,,,,,True,
4,CMG,AnVIL_CMG_Broad_Heart_Ware_WES,phs001272.v1.p1,,,,,,,,0,1,10,10,,,,,,,,True,
5,CMG,AnVIL_CMG_Broad_Muscle_Beggs_WES,phs001272.v1.p1,,,,,,,,0,1,109,109,,,,,,,,True,
6,CMG,AnVIL_CMG_Broad_Blood_Gazda_WGS,phs001272.v1.p1,,,,,,,,0,1,0,0,,,,,True,True,True,,True
7,CMG,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WES,phs001272.v1.p1,,,,,,,,0,1,31,31,,,,,,,,True,
8,CMG,AnVIL_CMG_Broad_Eye_Pierce_WES,phs001272.v1.p1,,,,,,,,0,1,602,602,,,,,,,,True,
9,CMG,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WGS,phs001272.v1.p1,,,,,,,,0,1,115,115,,,,,,,,True,


## summarize terra exceptions

> Extract the list of data transformation problems encountered [see more on dashboard exceptions](https://github.com/anvilproject/client-apis/wiki/dashboard-exceptions)

In [14]:
flattened = []
problems = set([problem for project in dashboard_data['projects'] for problem in project['problems']])
for problem in problems:
    projects = [project['project_id'] for project in dashboard_data['projects'] if problem in project['problems']]
    flattened.append([problem, ','.join(projects)])

# Print the data  (all rows, all columns)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'left')

df = pd.DataFrame(flattened)  
df.columns = ['problem', 'affected_workspaces']
df = df.style.set_properties(**{'text-align': 'left'})
df 

,problem,affected_workspaces
0,inconsistent_subject,"AnVIL_CMG_Broad_Muscle_Topf_WES,AnVIL_CMG_UWash_GRU,AnVIL_CMG_Broad_Blood_Gazda_WES,AnVIL_CCDG_WashU_CVD-NP-AI_Controls_VCControls_WGS"
1,inconsistent_entityName,"AnVIL_CCDG_WashU_AI_T1D_T1DGC_WGS,AnVIL_CCDG_NYGC_NP_Autism_SSC_WGS,AnVIL_CCDG_WashU_CVD-NP-AI_Controls_VCControls_WGS"
2,dbgap_sample_count_mismatch,"phs001272.v1.p1,phs001489.v1.p1,phs001642.v1.p1,phs001222.v1.p1,phs001227.v1.p1,phs001259.v1.p1,phs001543.v1.p1,phs001544.v1.p1,phs000160.v1.p1,phs001676.v1.p1,phs001502.v1.p1,phs001062.v4.p2,phs001180.v2.p1,phs000496.v1.p1,phs001395.v1.p1,phs001624.v1.p1,phs001545.v1.p1,phs000997.v4.p2,phs001398.v1.p1"
3,missing_subjects,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_Freeze2_VCFAggregation,AnVIL_CCDG_WashU_CVD_EOCAD_BioImage_WGS,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Freeze2_VCFs,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Broad_Deposit"
4,missing_sequence,"AnVIL_CMG_Broad_Muscle_KNC_WGS,ANVIL_CMG_Broad_Muscle_Laing_WES,AnVIL_CMG_Broad_Orphan_VCGS-White_WES,AnVIL_CMG_Broad_Muscle_Myoseq_WES,AnVIL_CMG_Broad_Heart_Ware_WES,AnVIL_CMG_Broad_Muscle_Beggs_WES,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WES,AnVIL_CMG_Broad_Eye_Pierce_WES,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WGS,AnVIL_CMG_Broad_Muscle_Topf_WES,AnVIL_CMG_Broad_Blood_Sankaran_WES,AnVIL_CMG_Broad_Brain_Walsh_WES,AnVIL_CMG_Broad_Muscle_Kang_WES,AnVIL_CMG_Broad_Kidney_Hildebrandt_WES,AnVIL_CMG_Broad_Muscle_OGrady_WES,AnVIL_CMG_Broad_Brain_Gleeson_WES,AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS,AnVIL_CMG_Broad_Orphan_Manton_WES,AnVIL_CMG_Broad_Heart_PCGC-Tristani_WGS,AnVIL_CMG_Broad_Muscle_KNC_WES,AnVIL_CMG_Broad_Heart_Seidman_WES,AnVIL_CMG_Broad_Muscle_Bonnemann_WGS,AnVIL_CMG_Broad_Muscle_Bonnemann_WES,AnVIL_CMG_Broad_Orphan_Manton_WGS,AnVIL_CMG_Broad_Muscle_Ravenscroft_WES,AnVIL_CMG_Broad_Orphan_VCGS-White_WGS,AnVIL_CMG_Broad_Muscle_Myoseq_WGS,AnVIL_CMG_Broad_Muscle_Kang_WGS,AnVIL_CMG_Broad_Eye_Pierce_WGS"
5,missing_accession,"AnVIL_CCDG_WashU_CVD_EOCAD_BioMe_WGS,AnVIL_CCDG_Broad_CVD_AFib_Penn_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_METSIM_WGS,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Broad_CVD_EOCAD_PROMIS_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_Duke_WGS,AnVIL_CCDG_Broad_CVD_AFib_Duke_WGS"
6,missing_samples,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_WashU_CVD_EOCAD_METSIM_WGS,AnVIL_CCDG_NYGC_NP_Autism_ACE2_DS-MDS_WGS,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Broad_AI_IBD_Cho_WGS,AnVIL_CCDG_NYGC_NP_Autism_SAGE_WGS,AnVIL_CCDG_Baylor_CVD_HemStroke_ERICH_WGS,AnVIL_CCDG_Baylor_CVD_EOCAD_SoL_WGS,AnVIL_CCDG_Freeze2_VCFs,AnVIL_CCDG_WashU_CVD_EOCAD_WashU-CAD_DS_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_Emory_WGS,AnVIL_CCDG_NYGC_NP_Autism_ACE2_GRU-MDS_WGS,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Baylor_CVD_ARIC,AnVIL_CCDG_Broad_Deposit,AnVIL_GTEx_V8_hg38"
7,missing_schema,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Freeze2_VCFs,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Broad_Deposit"
8,missing_blobs,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_Broad_NP_Epilepsy_DEUUPM_HMB_MDS_WES,AnVIL_CCDG_Broad_NP_Epilepsy_USAUPN_GRU_NPU_WES,AnVIL_ccdg_asc_ndd_daly_talkowski_AGRE-FEMF_asd_exome,AnVIL_CCDG_Broad_NP_Epilepsy_LEBABM_GRU_WES,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_WashU_CVD-NP-AI_Controls_VCControls_WGS,AnVIL_CCDG_Broad_Deposit"


## list consistent terra workspaces

In [16]:
# list consistent workspaces

df = pd.DataFrame([project['project_id'] for project in dashboard_data['projects'] if len(project['problems']) == 0])  
df.columns = ['workspace']
df = df.style.set_properties(**{'text-align': 'left'})
df 

,workspace
0,anvil_ccdg_broad_ai_ibd_daly_niddk_cho_wes
1,AnVIL_CCDG_Broad_NP_Epilepsy_ITAUBG_DS_EPI_NPU_MDS_WES
2,AnVIL_CCDG_Broad_NP_Epilepsy_USAMGH_MGBB_HMB_MDS_WES
3,AnVIL_CCDG_Broad_NP_Epilepsy_USACRW_EPI_ASZ_MED_MDS_WES
4,AnVIL_ccdg_asc_ndd_daly_talkowski_chung_asd_exome
5,AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS
6,AnVIL_CCDG_Broad_AI_IBD_Brant_DS-IBD_WGS
7,AnVIL_ccdg_asc_ndd_daly_talkowski_ac-boston_asd_exome
8,AnVIL_CCDG_Broad_NP_Epilepsy_AUSAUS_EPI_BA_ID_MDS_WES
9,anvil_ccdg_broad_ai_ibd_niddk_daly_silverberg_wes


## Issues/Questions arising from Gen3 PFB

## PFB contains gen3 projects without anvil(terra) project

In [17]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('/tmp/gen3-drs.sqlite')
cur = conn.cursor()

df = pd.read_sql_query("SELECT * from summary where anvil_project_id is null;", conn)
df

,project_id,anvil_project_id,subject_count,sample_count,sequencing_count,ga4gh_drs_uri_count
0,CCDG-phs001259-DS-CARD-MDS-GSO,None,2158,2159,0,0
1,CCDG-phs001398-GRU,None,496,496,0,0
2,CCDG-phs001487-DS-MULTIPLE_DISEASES-IRB-COL-NPU-RD,None,773,773,0,0
3,CCDG-phs001569-GRU,None,1136,1136,0,0
4,CCDG-phs001642-DS-GID,None,31,31,0,0
5,CCDG-phs001642-DS-IBD,None,199,199,0,0
6,CCDG-phs001642-GRU,None,1351,1351,0,0
7,CCDG-phs001642-HMB,None,1248,1248,0,0
8,CF-GTEx,None,981,47068,0,0
9,open_access-1000Genomes,None,3202,3202,0,0


## Not all terra projects found in Gen3

In [18]:
df = pd.read_sql_query("SELECT * from reconcile_counts where gen3_sample_id_count = 0;", conn)
df

,workspace_id,terra_sample_id_count,gen3_sample_id_count,terra_blob_count,gen3_drs_uri_count
0,1000G-high-coverage-2019,3202,0,9600,0
1,AnVIL_CCDG_Baylor_CVD_AFib_BioVU_WGS,1122,0,1122,0
2,AnVIL_CCDG_Baylor_CVD_AFib_Groningen_WGS,639,0,639,0
3,AnVIL_CCDG_Baylor_CVD_EOCAD_BioMe_WGS,1201,0,1201,0
4,AnVIL_CCDG_Broad_AI_IBD_Cho_WGS,344,0,688,0
5,AnVIL_CCDG_Broad_AI_IBD_McCauley_WGS,1096,0,2192,0
6,AnVIL_CCDG_Broad_AI_IBD_McGovern_WGS,1655,0,3310,0
7,AnVIL_CCDG_Broad_CVD_AF_BioVU_HMB_GSO_WES,5031,0,10062,0
8,AnVIL_CCDG_Broad_CVD_AFib_AFLMU_WGS,248,0,496,0
9,AnVIL_CCDG_Broad_CVD_AFib_Duke_WGS,123,0,246,0


## Terra / Gen3 samples count mismatch

In [19]:
df = pd.read_sql_query("SELECT * from reconcile_counts where gen3_sample_id_count > 0 and gen3_sample_id_count <> terra_sample_id_count;", conn)
df

,workspace_id,terra_sample_id_count,gen3_sample_id_count,terra_blob_count,gen3_drs_uri_count
0,AnVIL_CCDG_Broad_CVD_EOCAD_VIRGO_WGS,2159,2148,4318,4296
1,AnVIL_CMG_Broad_Brain_Gleeson_WES,1181,1176,2362,2352
2,AnVIL_CMG_Broad_Brain_Walsh_WES,767,766,2301,1532
3,AnVIL_CMG_Broad_Eye_Pierce_WES,602,412,1806,824
4,AnVIL_CMG_Broad_Heart_Seidman_WES,129,128,387,258
5,AnVIL_CMG_Broad_Muscle_Beggs_WES,109,87,327,174
6,AnVIL_CMG_Broad_Muscle_Bonnemann_WES,24,5,72,20
7,AnVIL_CMG_Broad_Muscle_Bonnemann_WGS,93,42,250,94
8,AnVIL_CMG_Broad_Muscle_KNC_WGS,14,7,42,14
9,AnVIL_CMG_Broad_Muscle_Myoseq_WES,1280,1248,3806,2532


## Terra / Gen3 blob/drs count alignment

In [20]:
df = pd.read_sql_query("SELECT * from reconcile_counts where terra_sample_id_count = gen3_sample_id_count and terra_blob_count = gen3_drs_uri_count;", conn)
df

,workspace_id,terra_sample_id_count,gen3_sample_id_count,terra_blob_count,gen3_drs_uri_count
0,ANVIL_CMG_Broad_Muscle_Laing_WES,31,31,62,62
1,AnVIL_CCDG_Broad_AI_IBD_Brant_DS-IBD_WGS,199,199,398,398
2,AnVIL_CCDG_Broad_AI_IBD_Brant_HMB_WGS,904,904,1808,1808
3,AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS,1351,1351,2702,2702
4,AnVIL_CCDG_Broad_AI_IBD_Newberry_WGS,31,31,62,62
5,AnVIL_CCDG_Broad_CVD_EOCAD_PROMIS_WGS,1136,1136,2272,2272
6,AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS,773,773,1546,1546
7,AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS,496,496,992,992


### Terra / Gen3 blob/drs count mismatch

In [21]:

df = pd.read_sql_query("SELECT * from reconcile_counts where terra_sample_id_count = gen3_sample_id_count and terra_blob_count <> gen3_drs_uri_count;", conn)
df

,workspace_id,terra_sample_id_count,gen3_sample_id_count,terra_blob_count,gen3_drs_uri_count
0,AnVIL_CMG_Broad_Eye_Pierce_WGS,35,35,105,70
1,AnVIL_CMG_Broad_Heart_Ware_WES,10,10,30,20
2,AnVIL_CMG_Broad_Kidney_Hildebrandt_WES,1222,1222,3304,2444
3,AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS,27,27,81,54
4,AnVIL_CMG_Broad_Muscle_KNC_WES,44,44,123,88
5,AnVIL_CMG_Broad_Muscle_Myoseq_WGS,10,10,30,40
6,AnVIL_CMG_Broad_Orphan_Manton_WGS,43,43,129,152
7,AnVIL_GTEx_V8_hg38,17382,17382,34764,55530


### Unexpected extra files not in terra [leafcutter, bigWig]

In [22]:
pd.set_option('max_colwidth', 256)
df = pd.read_sql_query("select * from terra_details where blob like '%GTEX-12KS4-1526-SM-5EQ6E%' ;", conn)
df

,workspace_id,subject_id,sample_id,blob
0,AnVIL_GTEx_V8_hg38,GTEX-12KS4,GTEX-12KS4-1526-SM-5EQ6E,gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_RNAseq_BAM_files/GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bam
1,AnVIL_GTEx_V8_hg38,GTEX-12KS4,GTEX-12KS4-1526-SM-5EQ6E,gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_RNAseq_BAM_files/GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bam.bai


In [23]:
df = pd.read_sql_query("select key, name, submitter_id  from vertices where submitter_id like '%GTEX-12KS4-1526-SM-5EQ6E%' ;", conn)
df

,key,name,submitter_id
0,0019be63-8121-4869-bc6f-2c2694d6aefd,sample,GTEX-12KS4-1526-SM-5EQ6E
1,4e090504-a493-44ee-a1d2-db8dd34f29d0,sequencing,GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bam.bai
2,663f2cb3-467d-46ec-8a05-908cfedc08e1,sequencing,GTEX-12KS4-1526-SM-5EQ6E.leafcutter.junc.gz
3,8abef4da-24c3-4698-9ee6-d2f71f0347ce,sequencing,GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bigWig
4,8bcad8c0-3122-42fd-bdaa-fc912002057a,sequencing,GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bam
5,deb14768-af73-44fd-891c-a4206ae94e0e,sequencing,GTEX-12KS4-1526-SM-5EQ6E.SJ.out.tab


## Unexpected Suffixes on gen3 identifiers [_RNASEQ_BAM_FILES, _RNASEQ_BIGWIG]

In [24]:
df = pd.read_sql_query("""select key, name,  json_extract(json, '$.object.submitter_id') as "gen3_submitter_id"   from vertices where gen3_submitter_id like '%_RNASEQ_BAM_FILES' or gen3_submitter_id like '%_BIGWIG'  limit 10;""", conn)
df

,key,name,gen3_submitter_id
0,000b3ad0-94a3-4caa-b1eb-c32d116fe410,sequencing,GTEX-14PJM-0011-R1a-SM-CNPO3.Aligned.sortedByCoord.out.patched.md.bam.bai_RNASEQ_BAM_FILES
1,004af85b-1aee-4287-9b22-9e0e514ff6a4,sequencing,GTEX-11DXW-0626-SM-5N9ER.Aligned.sortedByCoord.out.patched.md.bam_RNASEQ_BAM_FILES
2,00b132b9-8602-41b0-8c4e-9697195795f8,sequencing,GTEX-1A8G7-1126-SM-731ED.Aligned.sortedByCoord.out.patched.md.bigWig_RNASEQ_BIGWIG
3,00b9739d-1e69-418d-b04d-5f1b8664a624,sequencing,GTEX-ZF3C-1426-SM-4WWCD.Aligned.sortedByCoord.out.patched.md.bam_RNASEQ_BAM_FILES
4,00c6695d-6e70-432a-804e-05e4bd7beb59,sequencing,GTEX-1AMEY-1826-SM-72D5L.Aligned.sortedByCoord.out.patched.md.bam.bai_RNASEQ_BAM_FILES
5,00cc7d53-faa1-4d53-93e4-017643865cb2,sequencing,GTEX-OOBK-0126-SM-2YUND.Aligned.sortedByCoord.out.patched.md.bigWig_RNASEQ_BIGWIG
6,00daebe6-352e-4e7f-87ed-c207cc156d20,sequencing,GTEX-13NYS-3126-SM-5KLYV.Aligned.sortedByCoord.out.patched.md.bam_RNASEQ_BAM_FILES
7,00e143a2-6c6e-45c5-975d-8038bc553d15,sequencing,GTEX-14AS3-2226-SM-5S2OX.Aligned.sortedByCoord.out.patched.md.bigWig_RNASEQ_BIGWIG
8,00e4519c-3b46-4fb2-ac42-9f0ca931f7e9,sequencing,GTEX-12696-2826-SM-5FQTY.Aligned.sortedByCoord.out.patched.md.bam.bai_RNASEQ_BAM_FILES
9,010573d8-ce5d-4311-9f9d-46c48c6c561b,sequencing,GTEX-146FH-1226-SM-5NQB6.Aligned.sortedByCoord.out.patched.md.bigWig_RNASEQ_BIGWIG


## subjects without PFB `sequencing` record

In [25]:

df = pd.read_sql_query("select project_id, count(*) from flattened  where subject_id  in (select key from  subjects_missing_sequencing);", conn)
df

,project_id,count(*)
0,CF-GTEx,59316


## unexpected suffix on subject identifiers [_subject]

In [26]:
sql = """select project_id, count( distinct subject_id)   from flattened where subject_submitter_id  like '%_subject'"""
df = pd.read_sql_query(sql, conn)
df

,project_id,count( distinct subject_id)
0,CMG-Broad-GRU,6989


# Transform - write FHIR

In [27]:
write_fhir(workspace_names)

2020-12-18 02:08:14,081 ERROR    AnVIL_CMG_Broad_Blood_Gazda_WGS missing subject edges
2020-12-18 02:08:20,661 ERROR    AnVIL_CMG_Broad_Brain_Gleeson_WGS missing subject edges
2020-12-18 02:08:21,238 ERROR    AnVIL_CMG_Broad_Brain_Engle_WGS missing subject edges
2020-12-18 02:08:21,317 ERROR    AnVIL_CMG_Broad_Kidney_Pollak_WES missing subject edges
2020-12-18 02:08:21,318 ERROR    AnVIL_CMG_Broad_Orphan_Scott_WES missing subject edges
2020-12-18 02:08:22,016 ERROR    AnVIL_CMG_Broad_Blood_Fleming_WES missing subject edges
2020-12-18 02:08:36,295 ERROR    AnVIL_CCDG_Freeze2_VCFAggregation missing subject edges
2020-12-18 02:09:48,430 ERROR    AnVIL_CCDG_WashU_CVD_EOCAD_BioImage_WGS missing subject edges
2020-12-18 02:09:56,792 ERROR    AnVIL_CCDG_WASHU_PAGE missing subject edges
2020-12-18 02:11:13,126 ERROR    AnVIL_CCDG_Freeze2_VCFs missing subject edges
2020-12-18 02:11:46,647 ERROR    ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY missing subject edges
2020-12-18 02:12:42,490 ERROR    AnV

## "load" 

> Copy results to bucket

In [28]:
!gsutil cp -r /tmp/CCDG $WORKSPACE_BUCKET/CCDG
!gsutil cp -r /tmp/CMG $WORKSPACE_BUCKET/CMG
!gsutil cp -r /tmp/GTEx $WORKSPACE_BUCKET/GTEx
!gsutil cp -r /tmp/ThousandGenomes $WORKSPACE_BUCKET/ThousandGenomes
!gsutil cp /tmp/data_dashboard.json  $WORKSPACE_BUCKET
!gsutil cp /tmp/data_dashboard.tsv  $WORKSPACE_BUCKET


Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS/DocumentReference.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS/Practitioner.json [Content-Type=application/json]...
- [4 files][  1.3 MiB/  1.3 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS/Task.json [Content-

Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_VIRGO_WGS/DocumentReference.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_VIRGO_WGS/Practitioner.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_VIRGO_WGS/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_VIRGO_WGS/Task.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_VIRGO_WGS/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_EOCAD_VIRGO_WGS/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_USABCH_EPI_MUL_CON_MDS_WES/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_USABCH_EPI_MUL_CON_MDS_WES/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epile

Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Autism_ACE2_DS-MDS_WGS/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Autism_ACE2_DS-MDS_WGS/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMC_DS_NEURO_MDS_WES/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMC_DS_NEURO_MDS_WES/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMC_DS_NEURO_MDS_WES/DocumentReference.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMC_DS_NEURO_MDS_WES/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMC_DS_NEURO_MDS_WES/Task.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMC_DS_NEURO_MDS_WES/Organization.json [Content-Type=app

Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS/DocumentReference.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS/Practitioner.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS/Task.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_USAUPN_Marsh_GRU_WES/ResearchStudy.json [Content-Type=application/

Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_AFib_MPP_WGS/DocumentReference.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_AFib_MPP_WGS/Practitioner.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_AFib_MPP_WGS/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_AFib_MPP_WGS/Task.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_AFib_MPP_WGS/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_CVD_AFib_MPP_WGS/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_ccdg_asc_ndd_daly_talkowski_IBIS_asd_exome/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_ccdg_asc_ndd_daly_talkowski_IBIS_asd_exome/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_ccdg_asc_ndd_daly_talkowski_IBIS_asd_exome/Document

...

Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_NZLUTO_EPIL_BC_ID_MDS_WES/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Autism_State-Sanders_WGS/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Autism_State-Sanders_WGS/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Autism_State-Sanders_WGS/DocumentReference.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Autism_State-Sanders_WGS/Practitioner.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Autism_State-Sanders_WGS/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Autism_State-Sanders_WGS/Task.json [Content-Type=application/json]...
Copying file:///tmp/CCDG/AnVIL_CCDG_Broad_NP_Autism_State-Sanders_WGS/Organization.json [Content-Type=application/json]...
Copying file:/

Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Kang_WGS/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Kang_WGS/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/Practitioner.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/Observation.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/Task.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/Organization.json [Content-Type=appli

Copying file:///tmp/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/Task.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Orphan_Manton_WES/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Orphan_Manton_WES/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Orphan_Manton_WES/Practitioner.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Orphan_Manton_WES/Observation.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Orphan_Manton_WES/ResearchSubject.json [Content-Type=application/j

Copying file:///tmp/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Blood_Gazda_WES/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Blood_Gazda_WES/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Blood_Gazda_WES/DocumentReference.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Blood_Gazda_WES/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Blood_Gazda_WES/Task.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Blood_Gazda_WES/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Blood_Gazda_WES/Patient.json [Content-Type=application/json]...
Copying file:///tmp/

Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Bonnemann_WGS/Practitioner.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Bonnemann_WGS/Observation.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Bonnemann_WGS/ResearchSubject.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Bonnemann_WGS/Task.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Bonnemann_WGS/Organization.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Bonnemann_WGS/Patient.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Beggs_WES/ResearchStudy.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Beggs_WES/Specimen.json [Content-Type=application/json]...
Copying file:///tmp/CMG/AnVIL_CMG_Broad_Muscle_Beggs_WES/Practitioner.json [Content-Type=application/json]...


In [29]:
!gsutil ls -r $WORKSPACE_BUCKET

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/DocumentReference.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Task.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/data_dashboard.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/data_dashboard.tsv
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/export_2020-10-28T16_19_50.avro
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/export_2020-11-04T17_48_47.avro
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/export_2020-11-05T23_26_49.avro
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/r

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/DocumentReference.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS/Task.jso

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_JHU_WGS/DocumentReference.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_JHU_WGS/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_JHU_WGS/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_JHU_WGS/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_JHU_WGS/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_JHU_WGS/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_JHU_WGS/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_JHU_WGS/Task.json

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_AFib_MGH_WGS/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/DocumentReference.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS/Task.json

gs://f

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_BELULB_DS-EP-NPU_WES/Task.json

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_CANUTN_DS-EP_WES/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_CANUTN_DS-EP_WES/DocumentReference.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_CANUTN_DS-EP_WES/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_CANUTN_DS-EP_WES/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_CANUTN_DS-EP_WES/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_CANUTN_DS-EP_WES/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CCDG/AnVIL_CCDG_Broad_NP_Epilepsy_CANUTN_DS-EP_WES/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d

...

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_OGrady_WES/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_OGrady_WES/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_OGrady_WES/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_OGrady_WES/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_OGrady_WES/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_OGrady_WES/Task.json

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/Observation.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_Broad_Muscle_Ravenscroft_WES/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_UWash_GRU/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_UWash_GRU/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_UWash_GRU/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/AnVIL_CMG_UWash_GRU/Task.json

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/:

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/ANVIL_CMG_Broad_Muscle_Laing_WES/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/ANVIL_CMG_Broad_Muscle_Laing_WES/Observation.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/ANVIL_CMG_Broad_Muscle_Laing_WES/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/ANVIL_CMG_Broad_Muscle_Laing_WES/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/ANVIL_CMG_Broad_Muscle_Laing_WES/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-8

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WES/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WES/Task.json

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Kidney_Hildebran

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_Myoseq_WGS/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_Myoseq_WGS/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_Myoseq_WGS/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_Myoseq_WGS/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_Myoseq_WGS/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_Myoseq_WGS/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_Myoseq_WGS/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_Myoseq_WGS/Task.json

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Muscle_OGrady_WES/:
gs://fc-secure-d8ae6fb6-76be-43a4-87

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WES/Task.json

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/:
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/Observation.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/CMG/CMG/AnVIL_CMG_Broad_Orphan_VCGS-White_WGS

## you can load bucket contents into FHIR server
![image](https://user-images.githubusercontent.com/47808/102567204-e159de00-4095-11eb-883c-1f36e4790558.png)
![image](https://user-images.githubusercontent.com/47808/102567246-f46cae00-4095-11eb-8090-2fc28f1832e9.png)
